In [1]:
from data_generator.csv_data_processor import CSVDataProcessor

In [2]:
from utils.util_funcs import get_row_count

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL").getOrCreate()

csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")

# Read the CSV file
df = csv_reader.run()

# Show DataFrame
df.show()
df.createOrReplaceTempView("raw_healthcare_data")
get_row_count(df)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/23 21:26:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


runner func.


+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|                name|age|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|
+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|     Tiffany Ramirez| 81|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|
|         Ruben Burns| 35|  Male|        O+|           Asthma|       2023-06-01|     Diane Jacks

In [ ]:
# distinct_docs = spark.sql(f"select distinct Doctor from raw_healthcare_data")

In [ ]:
# distinct_docs.show()

In [ ]:
# distinct_patients_age = spark.sql("SELECT DISTINCT Name, Age FROM raw_healthcare_data")
# distinct_patients_name = spark.sql("SELECT DISTINCT Name FROM raw_healthcare_data")

In [ ]:
# get_row_count(distinct_patients_name)
# get_row_count(distinct_patients_age)


In [ ]:
# admission_types = spark.sql("SELECT DISTINCT Admission_Type FROM raw_healthcare_data")

In [ ]:
# get_row_count(admission_types)

In [4]:
from data_generator.admission_data_processor import AdmissionProcessor, create_new_admission_data

In [5]:
df_with_dob = df.withColumn("DOB", AdmissionProcessor.generate_dob()).drop("age")

In [6]:
df_with_dob.show()

+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------+
|                name|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|     DOB|
+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------+
|     Tiffany Ramirez|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|19561128|
|         Ruben Burns|  Male|        O+|           Asthma|       2023-06-01|

In [9]:
df_with_new_data = df_with_dob.withColumn("new_data", create_new_admission_data(df_with_dob["gender"], df_with_dob["DOB"]))

In [10]:
df_with_new_data.show()

+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------+--------------------+
|                name|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|     DOB|            new_data|
+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------+--------------------+
|     Tiffany Ramirez|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|19690524|{self_referral

Selected Admission Type: self_referral
Selected Sub-Admission Type: dermatology
Selected Stay Type: outpatient
Selected Tests: ['Patch tests', 'Skin scrapings', 'Skin biopsy', 'Blood tests']
Selected Admission Type: hospital_referral
Selected Sub-Admission Type: cancer
Selected Stay Type: day_patient
Selected Tests: ['MRI', 'Biopsy', 'CT scans', 'PET scans']
Selected Admission Type: emergency
Selected Sub-Admission Type: neurology
Selected Stay Type: inpatient
Selected Tests: ['Electroencephalogram (EEG)', 'Nerve conduction studies', 'Blood tests', 'MRI or CT scans of the brain']
Selected Admission Type: elective
Selected Sub-Admission Type: ophthalmology
Selected Stay Type: day_patient
Selected Tests: ['Visual field test', 'Eye exam', 'Retinal imaging']
Selected Admission Type: elective
Selected Sub-Admission Type: orthopedics
Selected Stay Type: inpatient
Selected Tests: ['CT scans', 'Bone scans']
Selected Admission Type: hospital_referral
Selected Sub-Admission Type: nephrology
Sele

In [14]:
df_with_new_data = (df_with_new_data.withColumn("ADMISSION_TYPE", df_with_new_data["new_data"]["ADMISSION_TYPE"]) 
                                    .withColumn("SUB_ADMISSION_TYPE", df_with_new_data["new_data"]["SUB_ADMISSION_TYPE"]) 
                                    .withColumn("STAY_TYPE", df_with_new_data["new_data"]["STAY_TYPE"]) 
                                    .withColumn("TESTS_SUGGESTED", df_with_new_data["new_data"]["TESTS_SUGGESTED"]).drop("new_data"))


In [15]:
df_with_new_data.show()

+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------------+--------------+-----------+------------+--------+--------------------+-----------+--------------------+
|                name|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|      ADMISSION_TYPE|discharge_date| medication|test_results|     DOB|  SUB_ADMISSION_TYPE|  STAY_TYPE|     TESTS_SUGGESTED|
+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------------+--------------+-----------+------------+--------+--------------------+-----------+--------------------+
|     Tiffany Ramirez|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          M

Selected Admission Type: emergency
Selected Sub-Admission Type: infectious_diseases
Selected Stay Type: inpatient
Selected Tests: ['PCR tests', 'Imaging (e.g., chest X-ray for pneumonia)']
Selected Admission Type: elective
Selected Sub-Admission Type: orthopedics
Selected Stay Type: day_patient
Selected Tests: ['MRI']
Selected Admission Type: pediatric_elective
Selected Sub-Admission Type: pediatric_orthopedics
Selected Stay Type: day_patient
Selected Tests: ['X-rays']
Selected Admission Type: hospital_referral
Selected Sub-Admission Type: neurology
Selected Stay Type: inpatient
Selected Tests: ['Blood tests']
Selected Admission Type: elective
Selected Sub-Admission Type: geriatrics
Selected Stay Type: outpatient
Selected Tests: ['Cognitive tests', 'Bone density scans', 'Blood tests', 'Comprehensive geriatric assessment']
Selected Admission Type: hospital_referral
Selected Sub-Admission Type: neurology
Selected Stay Type: inpatient
Selected Tests: ['Blood tests', 'Lumbar puncture', 'Ne